In [3]:
import os 
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
dj.conn()

[2023-03-02 16:52:09,527][INFO]: Connecting root@0.0.0.0:3306
[2023-03-02 16:52:09,559][INFO]: Connected root@0.0.0.0:3306


DataJoint connection (connected) root@0.0.0.0:3306

In [4]:
from datetime import datetime
from workflow.pipeline import ephys, probe, induction

In [ ]:
# Sample dataset directory structure for one subject, session, and probe
#    subjectAS001
#     ├── session2022-07-19
#     ├──── probe111
#     ├────── AS001_190722_204331.rhs
#     ├────── AS001_190722_204431.rhs
#     ├────── AS001_190722_204531.rhs
#     ├────── AS001_190722_204631.rhs
#     └────── AS001_190722_204731.rhs

In [14]:
# Insert into induction.OrganoidExperiment table (most upstream table)
induction.OrganoidExperiment.insert1(
        {
        "induction_id": "O6",
        "family": "Fam2_P",
        "line": "F2P6",
        "passage_id": "1",
        }
    ) 

induction.OrganoidExperiment()

induction_id de-identified code,family,line,passage_id
O6,Fam2_P,F2P6,1


### Ingest `probe` schema


In [9]:
probe.ProbeType.insert1(
    {
        "probe_type": "NeuroNexus-A4x16-Poly2-5mm",
    }
)

probe_config = {"probe_type": "A4x16-Poly2-5mm",
                "site_count_per_shank": 16,
                "col_spacing": None,
                "row_spacing": 20,
                "white_spacing": None,
                "col_count_per_shank": 1,
                "shank_count": 4,
                "shank_spacing": 100
                }

electrode_layouts = probe.build_electrode_layouts(**probe_config)
probe.ProbeType.Electrode.insert(electrode_layouts)

probe.Probe.insert1(
    {
        "probe": "111",
        "probe_type": "A4x16-Poly2-5mm",
        "probe_comment": "Example probe",
    }
)

channel_to_electrode_map = {
                            "D-000": 0,
                            "D-001": 1,
                            "D-002": 2,
                            "D-003": 3,
                            "D-004": 4,
                            "D-005": 5,
                            "D-006": 6,
                            "D-007": 7,
                            "D-008": 8,
                            "D-009": 9,
                            "D-010": 10,
                            "D-011": 11,
                            "D-012": 12,
                            "D-013": 13,
                            "D-014": 14,
                            "D-015": 15}
electrode_keys = [
    {"probe_type": "A4x16-Poly2-5mm", "channel": c, "electrode": e}
    for c, e in channel_to_electrode_map.items()
]

ephys.generate_electrode_config(
    probe_type="A4x16-Poly2-5mm", electrode_keys=electrode_keys
)

{'electrode_config_hash': UUID('6268a463-c131-8cc8-e14f-1d0b0eec0b69')}

In [10]:
display(probe.Probe())
display(probe.ProbeType())
display(probe.ProbeType.Electrode())
display(probe.ElectrodeConfig())
display(probe.ElectrodeConfig.Electrode())

probe unique identifier for this model of probe (e.g. serial number),probe_type e.g. neuropixels_1.0,probe_comment
111,A4x16-Poly2-5mm,dummy_probe


probe_type e.g. neuropixels_1.0
A4x16-Poly2-5mm


probe_type e.g. neuropixels_1.0,"electrode electrode index, starts at 0","shank shank index, starts at 0, advance left to right","shank_col column index, starts at 0, advance left to right","shank_row row index, starts at 0.",x_coord (um) x coordinate of the electrode within the probe.,y_coord (um) y coordinate of the electrode within the probe.
A4x16-Poly2-5mm,0,0,0,0,0.0,0.0
A4x16-Poly2-5mm,1,0,0,1,0.0,20.0
A4x16-Poly2-5mm,2,0,0,2,0.0,40.0
A4x16-Poly2-5mm,3,0,0,3,0.0,60.0
A4x16-Poly2-5mm,4,0,0,4,0.0,80.0
A4x16-Poly2-5mm,5,0,0,5,0.0,100.0
A4x16-Poly2-5mm,6,0,0,6,0.0,120.0
A4x16-Poly2-5mm,7,0,0,7,0.0,140.0
A4x16-Poly2-5mm,8,0,0,8,0.0,160.0
A4x16-Poly2-5mm,9,0,0,9,0.0,180.0


electrode_config_hash,probe_type e.g. neuropixels_1.0,electrode_config_name user friendly name
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,0-15


electrode_config_hash,probe_type e.g. neuropixels_1.0,"electrode electrode index, starts at 0","channel channel name fetched from raw data (e.g., A-001)"
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,0,D-000
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,1,D-001
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,2,D-002
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,3,D-003
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,4,D-004
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,5,D-005
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,6,D-006
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,7,D-007
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,8,D-008
6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,9,D-009


### Populate `ephys.RawData`


In [17]:
ephys.RawData.populate()
ephys.RawData()

induction_id de-identified code,start_time date and time of file creation,file_name name of the file,file_path
O6,2019-07-22 20:43:31,O6_190722_204331.rhs,=BLOB=
O6,2019-07-22 20:44:31,O6_190722_204431.rhs,=BLOB=
O6,2019-07-22 20:45:31,O6_190722_204531.rhs,=BLOB=
O6,2019-07-22 20:46:31,O6_190722_204631.rhs,=BLOB=
O6,2019-07-22 20:47:31,O6_190722_204731.rhs,=BLOB=


### Insert into `ephys.EphysSession`

In [19]:
ephys.EphysSession()

induction_id de-identified code,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),electrode_config_hash,session_type analysis method
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,111,6268a463-c131-8cc8-e14f-1d0b0eec0b69,lfp


In [21]:
key = induction.OrganoidExperiment.fetch1("KEY")

start_time = datetime.strptime("190722_204331", "%y%m%d_%H%M%S")
end_time = datetime.strptime("190722_204631", "%y%m%d_%H%M%S")

# ephys.EphysSession.insert1(
#         {**key, "insertion_number": 0, "start_time": start_time, 
#         "end_time": end_time, 
#         "probe": "111", 
#         "electrode_config_hash": "6268a463-c131-8cc8-e14f-1d0b0eec0b69", "session_type": "lfp"
#         } 
#     )

ephys.EphysSession()

induction_id de-identified code,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),electrode_config_hash,session_type analysis method
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,111,6268a463-c131-8cc8-e14f-1d0b0eec0b69,lfp


### Populate `ephys.EphysSessionInfo`


In [22]:
ephys.EphysSessionInfo.populate()
ephys.EphysSessionInfo()

induction_id de-identified code,insertion_number,start_time,end_time,attribute_name,attribute_blob
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,amp_settle_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,amplifier_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,board_adc_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,board_dac_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,board_dig_in_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,board_dig_out_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,charge_recovery_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,compliance_limit_channels,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,frequency_parameters,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,notes,=BLOB=


### Populate `ephys.LFP`


In [5]:
ephys.LFP.populate()
display(ephys.LFP())
display(ephys.LFP.Trace())

induction_id de-identified code,insertion_number,start_time,end_time,lfp_sampling_rate (Hz) down-sampled sampling rate.,header
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,2000.0,=BLOB=


induction_id de-identified code,insertion_number,start_time,end_time,electrode_config_hash,probe_type e.g. neuropixels_1.0,"electrode electrode index, starts at 0",lfp
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,11,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,5,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,15,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,0,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,2,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,10,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,8,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,7,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,14,=BLOB=
O6,0,2019-07-22 20:43:31,2019-07-22 20:46:31,6268a463-c131-8cc8-e14f-1d0b0eec0b69,A4x16-Poly2-5mm,6,=BLOB=
